In [16]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import jieba, re, time
jieba.set_dictionary('dict.txt.big')


In [17]:
## remove punctuation
def rm_punc(line, strip_all=True, user_punctuation = str()):
    import re
    if strip_all:
        if user_punctuation != str():
            print('[Warning]:user_punctuation ignored, if necessary, set strip_all=False')
        rule = re.compile(u"[^a-zA-Z0-9\u4e00-\u9fa5]")
        line = rule.sub('',line)
    else:
        import zhon.hanzi, string
        punctuation = zhon.hanzi.punctuation + string.punctuation + user_punctuation
        re_punctuation = "[{0}]".format(punctuation)
        rule = re.compile(re_punctuation)
        line = rule.sub(repl="", string=line)
    return line.strip()

In [29]:
## Ngram
def ngram(string, n = 2, user_stopwords= list()):
    if type(string) is not str:
        error_type = string.__class__.__name__
        raise TypeError('string must be str, not {0}.'.format(error_type))
    
    if type(n) is not int:
        error_type = n.__class__.__name__
        raise TypeError('n must be int, not {0}.'.format(error_type))
    
    if type(user_stopwords) not in (list, tuple, set, dict, None):
        error_type = user_stopwords.__class__.__name__
        raise TypeError('user_stopwords must be container type, not {0}.'.format(error_type))
    
    string_de_punc = rm_punc(string)
    for ptr in range(len(string_de_punc)-n+1):
        yield string_de_punc[ptr:(ptr+n)]

In [50]:
def word_freq(text_list,freq_lower_bound = 1):
    from pandas import DataFrame, Series
    from collections import Counter
    freq_table = dict(Counter(text_list).most_common())
    freq_table_trim = {k:v for k,v in freq_table.items() if v >= freq_lower_bound} # v >= 1 indicates rm those only appear once
    freq_table_df = DataFrame({'Freq':Series(freq_table_trim)})
    return freq_table_df

In [32]:
# Test
txt = '為什麼 聖結石 會被酸而 這群人 不會？'
seg = ngram(txt, n = 2)
[i for i in seg]

TypeError: unsupported operand type(s) for +: 'generator' and 'generator'

'type'

In [128]:
# a simple demo for loading dataset.
qa_pairs = []
documents = []
latent_stop = []
with open('data/Gossiping-QA-Dataset.txt', 'r', encoding='utf-8') as dataset:
    for i, line in enumerate(dataset):
        line = line.strip('\n')
        q,a = line.split('\t')
        for n in range(1, 5):
            [latent_stop.append(s) for s in ngram(q, n)]
        qa_pairs.append([q,a])
print("問答配對筆數:",len(qa_pairs))

WordFreq = word_freq(latent_stop, freq_lower_bound= 1000)
Que, Ans = zip(*qa_pairs)

問答配對筆數: 418202


In [131]:
# write ngram stopwords
ngram_stopwords = WordFreq.loc[WordFreq.Freq > 0.01*len(qa_pairs)].index.to_list()
keepwords = ['台灣', '中國','肥宅', '女生', '日本', '喜歡', '朋友', '自己']
[ngram_stopwords.remove(k) for k in keepwords if k in ngram_stopwords]
with open('stopwords.txt', 'w') as f:
    for ns in ngram_stopwords:
        f.write(ns)
        f.write('\n')

In [130]:
# read stopwords
with open('stopwords.txt', 'r') as f:
    stopwords= []
    for line in f:
        stopwords.append(re.sub(pattern = '\n', repl= '', string= line))
print(len(stopwords))
stopwords[1:10]

['沒有', '的八', '的八卦', '有沒', '有沒有', '什麼', '怎麼', '不是', '是不']

In [132]:
for i, q in enumerate(Que):
    q_seg = [seg for seg in jieba.cut(rm_punc(q)) if seg not in stopwords]  ## stopword建議不要有數字
    documents.append(TaggedDocument(q_seg, [i]))

In [133]:
# randomly print the qa pairs
import random

for i in range(10):
    idx = random.randint(0, len(qa_pairs))
    print("Q:", qa_pairs[idx][0])
    print("A:", qa_pairs[idx][1])

Q: 情人最後難免淪為朋友?
A: 不用擔心 連朋友都當不成的
Q: 有沒有佛母孔雀明王的八卦？
A: 面，之後談判，認孔雀為母,才被嗯嗯脫身出來，
Q: 有大客車執照去當技術員是什麼概念？
A: 開大客太累了 12hr起跳
Q: 清境農場為什麼有辦法開星巴克跟MOS
A: 清境的路還好吧 還沒有很難開 更往上才是難開
Q: 是討厭中共還是討厭中國人？
A: 中共以及替中共添磚塊的中國人跟台灣人 ^ ^
Q: 搞個肥宅照護管理是否有搞頭
A: 其實　我記得　國外　有
Q: 有沒有全家什麼最好喝的八卦？
A: 蘋果紅茶真的超好喝！
Q: 農歷新年是不是也該砍一砍？
A: 支持去中國化要徹底，身上有混到漢人基因的人要自殺
Q: 欸！領帶怎麼打！教一下！
A: 就這樣然後這樣之後那樣 就好了
Q: 現在房子是進場好時機？
A: 噁心天龍人又要弄臭桃園房價和物價了


In [134]:
jieba.lcut('肥宅謝師宴坐在教授旁邊會增加存在感嗎')

['肥宅', '謝師宴', '坐在', '教授', '旁邊', '會', '增加', '存在', '感', '嗎']

In [160]:
Que[0:20]

('為什麼 聖結石 會被酸而 這群人 不會？',
 '為什麼慶祝228會被罵可是慶端午不會？',
 '有沒有戰神阿瑞斯的八卦?',
 '理論與實務最脫節的系',
 '為什麼PTT這麼多人看棒球',
 '為什麼達摩祖師傳那麼好看?',
 '3D小畫家有人會畫嗎？',
 '對天龍人來說宜蘭4南部還４東部',
 '機車推出uber或計程機車會怎樣',
 '台中的龍邦世貿有人跳樓?',
 '抽到海陸會被笑娘炮兵嗎',
 '國高中的國文',
 '當初太陽花沒有留下垃圾也會被捧？',
 '世界上最難聽的國歌是哪首',
 '雙A的筆電到底可不可以帶進星巴克',
 '歐陽妮妮的男友看到她爸會有感覺嗎？',
 '有沒有imgurㄉ八卦？',
 '當年的潮物搖搖筆是噱頭嗎？',
 '肥宅初夜可以賣多少？',
 '+006是什麼地方打來的電話')

In [136]:
Ans[0:10]

('質感 劇本 成員 都差很多好嗎 不要拿腎結石來污辱這群人',
 '因為屈原不是台灣人，是楚國人。',
 '爵士就是阿瑞斯 男主角最後死了',
 '哪個系不脫節...你問最不脫節的簡單多了...',
 '肥宅才看棒球\u3000系壘一堆胖子',
 '達摩從頭到尾都是被動 (別人問他問題',
 '3D小當家有人會畫嗎',
 '他國事務..',
 '載到肥宅會很痛苦',
 '曾經當過全台第一高樓，可惜不到一年')

In [166]:
documents[0].words

['聖', '結石', '酸', '而', '這群']

In [ ]:
## Doc2Vec Failed ....
# start_time = time.time()
# # It takes much time ......
# model = Doc2Vec(documents,dm=1, dm_concat=1, vector_size=50, window=2, hs=1,alpha = 1e-4, sample = 1e-5, min_count=2, workers=4)
# # model.train(documents, total_examples=model.corpus_count, epochs=10) 
# end_time = time.time()
# print((end_time - start_time)/60)


# model.save('doc2vec_chatbot.model')
# model = Doc2Vec.load("doc2vec_chatbot.model")


## 檢驗正確性，隨機（結果不唯一）挑一篇trained dataset中的文黨，用模型重新infer，
## 再計算與trained dataset中文檔相似度，如果模型良好，相似度第一位應該就是挑出的文檔，
## words = u"聖結石 被酸" --> jieba
## infer_vector = model.infer_vector(['機車', '推出', 'uber', '或', '計程', '機車', '會', '怎樣'])

# infer_vector = model.infer_vector(['猴頭菇', '起來', '像', '雞肉', '口感'],alpha=0.025, steps=100)

# similar_documents = model.docvecs.most_similar([infer_vector], topn = 10)

# similar_documents

# for s in similar_documents:
#     print(s)
#     print(Que[s[0]])
#     print(documents[s[0]])


# model.most_similar('肥宅', topn = 20)